In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Đọc và chuẩn bị dữ liệu
df = pd.read_csv("jena_climate_2009_2016.csv")
df.columns = df.columns.str.strip()
features = ['T (degC)', 'p (mbar)', 'rh (%)', 'wv (m/s)']
data = df[features].dropna().values

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# 2. Tạo window dataset (seq_len=96, pred_len=24)
def create_dataset(data, seq_len=96, pred_len=24):
    X, Y = [], []
    for i in range(len(data) - seq_len - pred_len):
        X.append(data[i:i+seq_len])
        Y.append(data[i+seq_len:i+seq_len+pred_len])
    return np.array(X), np.array(Y)

seq_len, pred_len = 96, 24
X, Y = create_dataset(data_scaled, seq_len, pred_len)

# 3. Định nghĩa mô hình Autoformer rút gọn
class AutoformerSimple(nn.Module):
    def __init__(self, feature_size, d_model=64, nhead=4, num_layers=2, dropout=0.1):
        super(AutoformerSimple, self).__init__()
        self.input_proj = nn.Linear(feature_size, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.decoder = nn.Linear(d_model, feature_size)
    
    def forward(self, src):
        src = self.input_proj(src)  
        src = src.permute(1, 0, 2)  
        enc_out = self.encoder(src)  
        out = enc_out[-pred_len:]  
        out = out.permute(1, 0, 2)  
        out = self.decoder(out) 
        return out

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoformerSimple(feature_size=X.shape[2]).to(device)

# 4. Chuẩn bị DataLoader
from torch.utils.data import TensorDataset, DataLoader
batch_size = 32

X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
Y_tensor = torch.tensor(Y, dtype=torch.float32).to(device)

dataset = TensorDataset(X_tensor, Y_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 5. Huấn luyện mô hình
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

epochs = 10  # Bạn tăng lên để kết quả tốt hơn

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for xb, yb in dataloader:
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(dataset):.6f}")

# 6. Lấy dự báo trên toàn bộ dữ liệu
model.eval()
with torch.no_grad():
    preds = model(X_tensor).cpu().numpy()  # (num_samples, pred_len, features)

# Lấy dự báo bước đầu tiên làm đại diện (có thể thay đổi)
pred_first_step = preds[:, 0, :]

# 7. Phân cụm GMM trên dự báo bước đầu tiên
gmm = GaussianMixture(n_components=3, random_state=0)
gmm.fit(pred_first_step)
clusters = gmm.predict(pred_first_step)

# 8. Trực quan hóa phân cụm
df_vis = pd.DataFrame(pred_first_step, columns=features)
df_vis['Cluster'] = clusters

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_vis, x='T (degC)', y='rh (%)', hue='Cluster', palette='Set2', alpha=0.7)
plt.title("Phân cụm dự báo thời tiết với Autoformer + GMM")
plt.xlabel("Nhiệt độ (°C) (chuẩn hóa)")
plt.ylabel("Độ ẩm (%) (chuẩn hóa)")
plt.grid(True)
plt.legend(title="Cụm khí hậu")
plt.show()